<a href="https://colab.research.google.com/github/alfinmaulani/skripsi2/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install happytransformer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 45 kB 2.3 MB/s 
     |████████████████████████████████| 5.5 MB 13.3 MB/s 
     |████████████████████████████████| 441 kB 82.3 MB/s 
     |████████████████████████████████| 1.3 MB 47.2 MB/s 
     |████████████████████████████████| 163 kB 85.7 MB/s 
     |████████████████████████████████| 115 kB 80.1 MB/s 
     |████████████████████████████████| 95 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 77.2 MB/s 
     |████████████████████████████████| 127 kB 75.7 MB/s 
     |████████████████████████████████| 7.6 MB 48.9 MB/s 
     |████████████████████████████████| 115 kB 73.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      

In [8]:
import pandas as pd
import numpy as np
import happytransformer

In [ ]:
import happytransformer as ht

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
korpusfake='/content/drive/MyDrive/valid fake news/Fake.csv'
dffake=pd.read_csv(korpusfake)
# df.columns=['teks']

In [3]:
korpustrue='/content/drive/MyDrive/valid fake news/True.csv'
dftrue=pd.read_csv(korpustrue)
# df.columns=['teks']

In [5]:
dffake.iloc[0,1]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev